In [1]:
import os
import time
import signal
import tracemalloc
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from contextlib import contextmanager
import numpy as np
from TDD.TDD import Ini_TDD,Clear_TDD,set_index_order
from TDD.TDD_Q import cir_2_tn,add_trace_line,add_inputs,add_outputs
#get_real_qubit_num
from TDD.TN import Index,Tensor,TensorNetwork

In [2]:
def count_c_gates(circuit):
    # Count the number of 2-qubit CNOT gates
    num_cnot_gates = sum(1 for instr, _, _ in circuit.data if instr.num_qubits == 2)
    return num_cnot_gates

In [3]:


# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Context manager to handle timeouts
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutException("Simulation timed out")
    
    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        # Cancel the alarm
        signal.alarm(0)

def benchmark_circuits(directory,aer_method="statevector"):
    # List all QASM files in the directory
    qasm_files = [file for file in os.listdir(directory) if file.endswith(".qasm")]

    # Create a file to store benchmark results
    results_file = open(aer_method+"_benchmark.txt", "w")
    results_file.write("Circuit Name,Time (s),Number of Qubits,Number of Gates,Number of Controlled Gates,Transpile Time\n")

    for qasm_file in qasm_files:
        try:
            # Load the QASM file
            qasm_file_path = os.path.join(directory, qasm_file)
            circuit = QuantumCircuit.from_qasm_file(qasm_file_path)

            # Transpile the circuit for simulation
            backend = AerSimulator(method=aer_method)
            start_time = time.time()
            qc=transpile(circuit,backend=backend)
            end_time = time.time()
            transpile_time = end_time - start_time

            # Get basic circuit information
            num_qubits = circuit.num_qubits
            num_gates = sum(circuit.count_ops().values())
            num_c_gates = count_c_gates(circuit)

            # Measure time to run simulation
            with timeout(600):  # Set timeout to 600 seconds (10 minutes)
                start_time = time.time()
                job=backend.run(qc)
                result=job.result()
                end_time = time.time()
                runtime = end_time - start_time

                # Write benchmark results to file
                results_file.write(f"{qasm_file},{runtime},{num_qubits},{num_gates},{num_c_gates},{transpile_time}\n")

        except TimeoutException:
            # If the simulation times out, write "TO" as runtime
            results_file.write(f"{qasm_file},TO,NA,NA,NA\n")
        except Exception as e:
            # Handle other exceptions gracefully
            print(f"Error processing {qasm_file}: {e}")

    results_file.close()

# Provide the directory containing QASM files
directory_path = "./Benchmarks"
methods=["statevector","matrix_product_state"]
for method in methods:
    benchmark_circuits(directory_path,method)


Error processing bv_30.qasm: 'Number of qubits (30) in circuit-170 is greater than maximum (29) in the coupling_map'
Error processing bv_50.qasm: 'Number of qubits (50) in circuit-187 is greater than maximum (29) in the coupling_map'
Error processing bv_70.qasm: 'Number of qubits (70) in circuit-194 is greater than maximum (29) in the coupling_map'
Error processing bv_60.qasm: 'Number of qubits (60) in circuit-215 is greater than maximum (29) in the coupling_map'
Error processing bv_40.qasm: 'Number of qubits (40) in circuit-224 is greater than maximum (29) in the coupling_map'
Error processing bv_80.qasm: 'Number of qubits (80) in circuit-235 is greater than maximum (29) in the coupling_map'
Error processing bv_100.qasm: 'Number of qubits (100) in circuit-254 is greater than maximum (29) in the coupling_map'
Error processing bv_90.qasm: 'Number of qubits (90) in circuit-267 is greater than maximum (29) in the coupling_map'
Error processing bv_70.qasm: 'Number of qubits (70) in circuit

In [4]:


# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Context manager to handle timeouts
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutException("Simulation timed out")
    
    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        # Cancel the alarm
        signal.alarm(0)

def benchmark_circuits(directory):
    # List all QASM files in the directory
    qasm_files = [file for file in os.listdir(directory) if file.endswith(".qasm")]

    # Create a file to store benchmark results
    results_file = open("TDD_benchmark.txt", "w")
    results_file.write("Circuit Name,Time (s),Number of Qubits,Number of Gates,Number of Controlled Gates,Initialisation Time\n")

    for qasm_file in qasm_files:
        try:
            # Load the QASM file
            qasm_file_path = os.path.join(directory, qasm_file)
            circuit = QuantumCircuit.from_qasm_file(qasm_file_path)

            # Get basic circuit information
            num_qubits = circuit.num_qubits
            num_gates = sum(circuit.count_ops().values())
            num_c_gates = count_c_gates(circuit)


            # create the tensor network
            start_time = time.time()
            input_s=[0]*num_qubits
            tn,all_indexs=cir_2_tn(circuit)
            add_inputs(tn,input_s,num_qubits)
            initialisation_time=time.time()-start_time
            # Measure time to run simulation
            with timeout(600):  # Set timeout to 600 seconds (10 minutes)
                start_time = time.time()
                Ini_TDD(index_order=all_indexs)
                tdd=tn.cont(optimizer='tree_decomposition')
                end_time = time.time()
                runtime = end_time - start_time

                # Write benchmark results to file
                results_file.write(f"{qasm_file},{runtime},{num_qubits},{num_gates},{num_c_gates},{initialisation_time}\n")

        except TimeoutException:
            # If the simulation times out, write "TO" as runtime
            results_file.write(f"{qasm_file},TO,NA,NA,NA\n")
        except Exception as e:
            # Handle other exceptions gracefully
            print(f"Error processing {qasm_file}: {e}")

    results_file.close()

# Provide the directory containing QASM files
directory_path = "./Benchmarks"
benchmark_circuits(directory_path)


In [5]:
# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Context manager to handle timeouts
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutException("Simulation timed out")
    
    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        # Cancel the alarm
        signal.alarm(0)

def benchmark_circuits(directory,aer_method="statevector"):
    # Start tracing memory allocations
    tracemalloc.start()

    # List all QASM files in the directory
    qasm_files = [file for file in os.listdir(directory) if file.endswith(".qasm")]

    # Create a file to store benchmark results
    results_file = open(aer_method+"_Benchmark_Memory.txt", "w")
    results_file.write("Circuit Name,Time (s),Memory Peak (MB),Number of Qubits,Number of Gates,Number of Controlled Gates\n")

    for qasm_file in qasm_files:
        try:
            # Load the QASM file
            qasm_file_path = os.path.join(directory, qasm_file)
            circuit = QuantumCircuit.from_qasm_file(qasm_file_path)


            # Transpile the circuit for simulation
            backend = AerSimulator(method=aer_method)
            qc=transpile(circuit,backend=backend)

            # Get basic circuit information
            num_qubits = circuit.num_qubits
            num_gates = sum(circuit.count_ops().values())
            num_cnot_gates = count_c_gates(circuit)

            # Measure time to run simulation
            with timeout(600):  # Set timeout to 600 seconds (10 minutes)
                start_time = time.time()
                job=backend.run(qc)
                result=job.result()
                end_time = time.time()
                runtime = end_time - start_time

                # Measure peak memory usage
                peak = tracemalloc.get_traced_memory()[1] / (1024 * 1024)  # in MB
                tracemalloc.clear_traces()

                # Write benchmark results to file
                results_file.write(f"{qasm_file},{runtime},{peak},{num_qubits},{num_gates},{num_cnot_gates}\n")

        except TimeoutException:
            # If the simulation times out, write "TO" as runtime and memory peak
            results_file.write(f"{qasm_file},TO,TO,NA,NA,NA\n")
        except Exception as e:
            # Handle other exceptions gracefully
            print(f"Error processing {qasm_file}: {e}")

    # Stop tracing memory allocations
    tracemalloc.stop()

    results_file.close()

# Provide the directory containing QASM files
directory_path = "./Benchmarks"
methods=["statevector","matrix_product_state"]
for method in methods:
    benchmark_circuits(directory_path,method)


Error processing bv_30.qasm: 'Number of qubits (30) in circuit-511 is greater than maximum (29) in the coupling_map'
Error processing bv_50.qasm: 'Number of qubits (50) in circuit-528 is greater than maximum (29) in the coupling_map'
Error processing bv_70.qasm: 'Number of qubits (70) in circuit-535 is greater than maximum (29) in the coupling_map'
Error processing bv_60.qasm: 'Number of qubits (60) in circuit-556 is greater than maximum (29) in the coupling_map'
Error processing bv_40.qasm: 'Number of qubits (40) in circuit-565 is greater than maximum (29) in the coupling_map'
Error processing bv_80.qasm: 'Number of qubits (80) in circuit-576 is greater than maximum (29) in the coupling_map'
Error processing bv_100.qasm: 'Number of qubits (100) in circuit-595 is greater than maximum (29) in the coupling_map'
Error processing bv_90.qasm: 'Number of qubits (90) in circuit-608 is greater than maximum (29) in the coupling_map'
Error processing bv_70.qasm: 'Number of qubits (70) in circuit

In [6]:


# Define a custom exception for timeout
class TimeoutException(Exception):
    pass

# Context manager to handle timeouts
@contextmanager
def timeout(seconds):
    def timeout_handler(signum, frame):
        raise TimeoutException("Simulation timed out")
    
    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        # Cancel the alarm
        signal.alarm(0)

def benchmark_circuits(directory):
    # Start tracing memory allocations
    tracemalloc.start()
    # List all QASM files in the directory
    qasm_files = [file for file in os.listdir(directory) if file.endswith(".qasm")]

    # Create a file to store benchmark results
    results_file = open("TDD_Benchmark_Memory.txt", "w")
    results_file.write("Circuit Name,Time (s),Memory Peak (MB),Number of Qubits,Number of Gates,Number of Controlled Gates\n")

    for qasm_file in qasm_files:
        try:
            # Load the QASM file
            qasm_file_path = os.path.join(directory, qasm_file)
            circuit = QuantumCircuit.from_qasm_file(qasm_file_path)

            # Get basic circuit information
            num_qubits = circuit.num_qubits
            num_gates = sum(circuit.count_ops().values())
            num_c_gates = count_c_gates(circuit)


            # create the tensor network
            start_time = time.time()
            input_s=[0]*num_qubits
            tn,all_indexs=cir_2_tn(circuit)
            add_inputs(tn,input_s,num_qubits)
            initialisation_time=time.time()-start_time
            # Measure time to run simulation
            with timeout(600):  # Set timeout to 600 seconds (10 minutes)
                start_time = time.time()
                Ini_TDD(index_order=all_indexs)
                tdd=tn.cont(optimizer='tree_decomposition')
                end_time = time.time()
                runtime = end_time - start_time
                
                # Measure peak memory usage
                peak = tracemalloc.get_traced_memory()[1] / (1024 * 1024)  # in MB
                tracemalloc.clear_traces()

                # Write benchmark results to file
                results_file.write(f"{qasm_file},{runtime},{peak},{num_qubits},{num_gates},{num_c_gates}\n")


        except TimeoutException:
            # If the simulation times out, write "TO" as runtime
            results_file.write(f"{qasm_file},TO,NA,NA,NA\n")
        except Exception as e:
            # Handle other exceptions gracefully
            print(f"Error processing {qasm_file}: {e}")

    results_file.close()

# Provide the directory containing QASM files
directory_path = "./Benchmarks"
benchmark_circuits(directory_path)
